In [1]:
import pandas as pd
from helpers import read_csv, load_pickled_data, pickle_data
import os
import matplotlib.pyplot as plt 
import plotly
from helpers import create_nx_graph
import plotly.graph_objects as go
import os, sys

from helpers import bfs, get_class_pairs, choose_node_based_on_neighbors, load_pickled_data
import numpy as np
import networkx as nx

importances_dict:dict = load_pickled_data("feature_importances.pkl")
feat_occurences:dict = load_pickled_data('frequencies.pkl')
print(feat_occurences)
importances = {}
names = []
i = 0
for key, value in importances_dict.items():
    importances[i] = value
    names.append(key)


keep_cols = ["txId", 'Time step'] + names
TIMESTEPS =[13]#, 42, 35, 32, 29, 22, 9 ,20]

DATASET_BASE = "dataset"
PICKLED_TIMESTEPS = os.path.join("exports", "split_on_timestep.pkl")
txs_features:pd.DataFrame =  read_csv(os.path.join(DATASET_BASE, "elliptic_txs_features.csv"), ret_Dataframe=True)[keep_cols]



{'Local_feature_2': {46: 1, 45: 1, 15: 330, 25: 238, 44: 18, 28: 86, 34: 189, 21: 37, 30: 925, 38: 12, 37: 572, 14: 736, 36: 119, 8: 70, 23: 214, 32: 26, 29: 112, 13: 19, 39: 2, 42: 4, 18: 329, 6: 13, 43: 41, 5: 175, 26: 54, 33: 20, 9: 31, 31: 18, 4: 5, 3: 18, 24: 7, 41: 6, 35: 3, 16: 22, 7: 18, 27: 7, 20: 3, 11: 8, 10: 11, 40: 1, 12: 6, 1: 7, 2: 1, 22: 1}, 'Local_feature_5': {13: 14, 1: 3315, 2: 293, 0: 677, 4: 145, 10: 20, 9: 16, 6: 8, 12: 3, 5: 13, 8: 6, 7: 6}, 'Local_feature_18': {8: 82, 5: 27, 14: 212, 19: 708, 9: 89, 1: 694, 3: 42, 28: 830, 23: 385, 38: 86, 40: 101, 21: 338, 34: 22, 36: 19, 26: 72, 50: 35, 13: 64, 45: 11, 22: 154, 2: 1, 20: 138, 6: 12, 49: 8, 32: 9, 41: 24, 25: 59, 52: 20, 31: 10, 12: 32, 35: 47, 39: 15, 42: 12, 15: 25, 51: 15, 10: 31, 4: 15, 46: 24, 37: 3, 7: 6, 47: 8, 48: 1, 11: 4, 18: 1, 33: 2, 53: 4, 44: 8, 54: 1, 17: 1, 24: 1, 29: 1, 30: 3, 43: 2, 16: 1, 27: 1}, 'Local_feature_41': {0: 2480, 21: 453, 17: 277, 16: 97, 19: 151, 5: 77, 9: 36, 28: 315, 33: 82, 1

In [2]:
txs_features = txs_features[keep_cols]
txs_classes:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_classes.csv"), ret_Dataframe=True)
txs_edgelist:pd.DataFrame = read_csv(os.path.join(DATASET_BASE, "elliptic_txs_edgelist.csv"), ret_Dataframe=True)


merge_feats_class = pd.merge(txs_features, txs_classes, on='txId').dropna()
merge_feats_class.shape
print(f"Total Nodes df shape: {txs_features.shape}")
print(f"total edges df shape: {txs_edgelist.shape}")
#timestep_graphs = load_pickled_data(PICKLED_TIMESTEPS)
timestep_dataframe = merge_feats_class.loc[merge_feats_class['Time step'].isin(TIMESTEPS)].reset_index(drop=True)
timestep_edges = txs_edgelist[txs_edgelist['txId1'].isin(timestep_dataframe['txId'])].reset_index(drop=True)

from helpers import discretize_per_column
freq, timestep_dataframe = discretize_per_column(timestep_dataframe, 7, 0 True)
timestep_cols = timestep_dataframe.columns
timestep_list = timestep_dataframe.values.tolist()

edge_list = timestep_edges.values.tolist()

from Graph import Graph, Node
from visualizations import visualize_graph_from_list, visualize_graph_df

graph_of_interest = Graph()
graph_of_interest.create_from_lists(timestep_cols, timestep_list, edge_list)
graph_of_interest.feature_importances =  importances_dict
graph_of_interest.feature_occurences = feat_occurences
pickle_data("working_graph.pkl", graph_of_interest, True)

# g:nx.DiGraph = create_nx_graph(timestep_dataframe, timestep_edges)

Total Nodes df shape: (203769, 31)
total edges df shape: (234355, 2)


TypeError: discretize_per_column() missing 1 required positional argument: 'min_s'

In [ ]:
timestep_dataframe

,txId,Time step,Local_feature_2,Local_feature_5,Local_feature_18,Local_feature_41,Local_feature_47,Local_feature_52,Local_feature_53,Local_feature_55,...,Local_feature_90,Local_feature_91,Aggregate_feature_43,Aggregate_feature_44,Aggregate_feature_49,Aggregate_feature_52,fees,size,num_output_addresses,class
0,327038,13,24,8,3,0,1,4,4,6,...,7,24,6,17,0,19,24,21,8,3
1,459148,13,23,8,3,10,13,4,13,13,...,7,24,2,17,0,19,23,20,8,2
2,461167,13,8,1,5,0,1,9,6,8,...,5,17,4,14,11,7,8,6,2,3
3,611515,13,14,2,6,0,1,1,0,1,...,7,18,3,16,8,16,14,7,3,3
4,736448,13,22,8,4,0,1,5,4,6,...,7,24,6,17,0,18,22,19,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,340080732,13,16,1,8,9,8,6,4,6,...,7,18,8,17,9,4,16,9,2,3
4512,340080737,13,16,1,8,9,8,6,4,6,...,5,17,8,14,9,5,16,9,2,3
4513,340080740,13,16,1,8,9,8,6,4,6,...,5,17,8,14,6,3,16,10,2,3
4514,340081441,13,7,1,8,16,15,14,11,13,...,8,24,8,14,6,5,7,9,2,3
